Env: ocean + dwave <br>
Auth: sorin_moldoveanu <br>
Referfences: https://docs.dwavesys.com/docs/latest<br>
             http://docs.ocean.dwavesys.com/en/latest/overview/solving_problems.html<br>
           

__Sample Portfolio__

 problem: Markovic portfolio<br>
 
 $min_{x \in \{0, 1\}^n} q x^T \Sigma x -\mu^Tx$ <br>
 subject to<br>
 $1^Tx=B$
 

In [30]:
#imports

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import numpy as np
import dimod
#from sklearn.datasets import make_spd_matrix

n=5
penalty=5
budget=int(n/2) 
q=0.5
mu = np.random.uniform(size=n, low=0, high=1)
sigma = np.random.rand(n,n) #make_spd_matrix(n)
e=np.ones(n)
E = np.matmul(np.asmatrix(e).T, np.asmatrix(e))
offset_H = - np.dot(mu, e)/2 + penalty*budget**2 - budget*n*penalty + n**2*penalty/4 + q/4*np.dot(e, np.dot(sigma, e))
offset_Q = -penalty*budget**2

qubo = [[0]*n for i in range(n)]
for i in range(n):
    qubo[i][i]=  -mu[i] + 2*penalty*budget
for i in range(n):
    for j in range(i+1,n):
        qubo[i][j] = -penalty + q*sigma[i][j]


print('QUBO')
for row in qubo:
    print(' '.join([str(elem) for elem in row]))
print('\n')

biases={('x0','x0'):qubo[0][0],('x1','x1'):qubo[1][1],('x2','x2'):qubo[2][2],('x3','x3'):qubo[3][3],
        ('x4','x4'):qubo[4][4]}
#biases={('x0','x0'):qubo[0][0],('x1','x1'):qubo[1][1],('x2','x2'):qubo[2][2],('x3','x3'):qubo[3][3]}

couplers={('x0','x1'):qubo[0][1],('x0','x2'):qubo[0][2],('x0','x3'):qubo[0][3],('x0','x4'):qubo[0][4],
          ('x1','x2'):qubo[1][2],('x1','x3'):qubo[1][3],('x1','x4'):qubo[1][4],
          ('x2','x3'):qubo[2][3],('x2','x4'):qubo[2][4],
          ('x3','x4'):qubo[3][4]}

#couplers={('x0','x1'):qubo[0][1],('x0','x2'):qubo[0][2],('x0','x3'):qubo[0][3],
#          ('x1','x2'):qubo[1][2],('x1','x3'):qubo[1][3],
#          ('x2','x3'):qubo[2][3]}

quis = [[0]*n for i in range(n)]
for i in range(n):
    quis[i][i]=  mu[i]/2 + budget*penalty - n*penalty/2 - q/2*sigma[i][i]
for i in range(n):
    for j in range(i+1,n):
        quis[i][j] = penalty/4 + q/4*sigma[i][j]
lin_h={('x0','x0'):quis[0][0],('x1','x1'):quis[1][1],('x2','x2'):quis[2][2],('x3','x3'):quis[3][3],
       ('x4','x4'):quis[4][4]}
quad_J={('x0','x1'):quis[0][1],('x0','x2'):quis[0][2],('x0','x3'):quis[0][3],('x0','x4'):quis[0][4],
          ('x1','x2'):quis[1][2],('x1','x3'):quis[1][3],('x1','x4'):quis[1][4],
          ('x2','x3'):quis[2][3],('x2','x4'):quis[2][4],
          ('x3','x4'):quis[3][4]}
print('ISING')
for row in quis:
    print(' '.join([str(elem) for elem in row]))
print('\n')

bqm=dimod.BinaryQuadraticModel(biases,couplers,offset_Q,dimod.BINARY)
#Q=dict(biases)
#Q.update(couplers)
Q=dict(lin_h)
Q.update(quad_J)
#print('Q:',Q)
#print('\navail_qbubits:',DWaveSampler().nodelist[0:8]) 


QUBO
19.25679290017624 -4.708991867008319 -4.6747990440603955 -4.607279672134617 -4.76833136908724
0 19.148948610891445 -4.67023161933787 -4.8031121726290715 -4.516346157374592
0 0 19.595189393266498 -4.6841940259729204 -4.70960976139324
0 0 0 19.392458495976697 -4.5531782831481555
0 0 0 0 19.559405511262703


ISING
-2.203087367215379 1.3227520332479203 1.3313002389849011 1.3481800819663456 1.30791715772819
0 -2.104288767917947 1.3324420951655325 1.2992219568427321 1.370913460656352
0 0 -2.3642308985026195 1.3289514935067699 1.32259755965169
0 0 0 -2.3414620717799304 1.3617054292129611
0 0 0 0 -2.4660565566778927




In [27]:
response=dimod.ExactSolver().sample(bqm)
#print(response)
#for sample, energy in response.data(['sample', 'energy']): print(sample,energy)
print(response.record)

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -20.        , 1)
 ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0],  -0.26808342, 1)
 ([1, 1, 0, 0, 0, 0, 0, 0, 0, 0],  18.91534126, 1) ...
 ([1, 1, 0, 0, 0, 0, 0, 0, 0, 1],  18.91534126, 1)
 ([1, 0, 0, 0, 0, 0, 0, 0, 0, 1],  -0.26808342, 1)
 ([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], -20.        , 1)]


In [31]:
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q, num_reads=600)
#response = EmbeddingComposite(DWaveSampler()).sample_qubo(Q, num_reads=300)
#for datum in response.data(['sample', 'energy', 'num_occurrences']):     
#   print(datum.sample,"Energy:", datum.energy, "Occurrences: ", datum.num_occurrences)
print(response)

   x0 x1 x2 x3 x4   energy num_oc. chain_.
0   0  0  1  0  1 -3.50769     172     0.0
1   0  0  0  1  1 -3.445813     107     0.0
2   0  0  1  1  0 -3.376741     113     0.0
3   1  0  0  0  1 -3.361227      51     0.0
4   1  0  1  0  0 -3.236018      47     0.0
5   0  1  0  0  1 -3.199432      15     0.0
6   1  0  0  1  0 -3.196369      32     0.0
7   0  0  1  1  1 -3.158495      11     0.0
8   0  1  0  1  0 -3.146529      18     0.0
9   0  1  1  0  0 -3.136078      23     0.0
10  1  0  0  1  1 -2.992803       3     0.0
11  1  1  0  0  0 -2.984624       4     0.0
12  0  1  1  0  1 -2.908623       1     0.0
13  0  1  1  1  0 -2.849366       1     0.0
14  1  1  0  0  1 -2.77185       1     0.0
15  1  1  0  1  0 -2.678684       1     0.0
['BINARY', 16 rows, 600 samples, 5 variables]


In [32]:
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_ising(lin_h,quad_J, num_reads=600)
print(response.record)

[([ 1,  1,  1,  1,  1,  1, -1,  1,  1,  1], -8.80380325, 236, 0.3)
 ([ 1,  1,  1,  1,  1,  1,  1,  1,  1, -1], -8.87941137, 363, 0.3)
 ([ 1, -1,  1,  1,  1,  1, -1,  1,  1,  1], -4.59522571,   1, 0.3)]
